In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [5]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Load data

In [6]:
eps_old_trap = np.load("epslion_old_trap_20210812.npy")
eps_new_trap = np.load("epslion_new_trap.npy")
eps_new_trap2 = np.load("epslion_new_trap_bead_20210830.npy")

In [7]:
_,ax = plt.subplots(figsize=(9.4,4))
time_hr_new = np.arange(len(eps_new_trap.T))*10*2/3600
time_hr_new2 = np.arange(len(eps_new_trap2.T))*10*2/3600
time_hr_old = np.arange(len(eps_old_trap.T))*10*2/3600
ax.errorbar(time_hr_old, eps_old_trap[0,:], yerr=eps_old_trap[1,:], fmt='o', label='A parameter', alpha=0.3)
ax.errorbar(time_hr_new, eps_new_trap[0,:], yerr=eps_new_trap[1,:], fmt='o', label='A parameter', alpha=0.3)
ax.errorbar(time_hr_new2, eps_new_trap2[0,:], yerr=eps_new_trap2[1,:], fmt='o', label='A parameter', alpha=0.3)
ax.plot(time_hr_old, time_hr_old*0, 'r--', zorder=3)
ax.legend()
ax.set(xlabel='Time [hr.]', ylabel=r'$\hat{\epsilon}$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'Time [hr.]'), Text(0, 0.5, '$\\hat{\\epsilon}$')]

In [8]:
thresh = 0.015
thresh2 = 0.015*10
data3_filt = np.array([dd for dd in eps_old_trap.T if dd[0]<thresh if dd[0]>-thresh])
data3_filt_new = np.array([dd for dd in eps_new_trap.T if dd[0]<thresh if dd[0]>-thresh])
data3_filt_new2 = np.array([dd for dd in eps_new_trap2.T if dd[0]<thresh2 if dd[0]>-thresh2])

print(len(eps_new_trap.T), len(data3_filt_new))
print(len(eps_new_trap2.T), len(data3_filt_new2))
print(len(eps_old_trap.T), len(data3_filt))

5074 5031
4974 4959
16607 16537


In [9]:
eps_new_trap.shape, data3_filt.shape

((2, 5074), (16537, 2))

In [10]:
eps_mean_old = []
eps_std_old = []

bin_size = 100
bins = len(eps_old_trap[0,:])//bin_size
for i in range(bins):
    eps_mean_old.append(np.mean(data3_filt.T[0,:][i*bin_size:(i+1)*bin_size]))
    eps_std_old.append(np.std(data3_filt.T[1,:][i*bin_size:(i+1)*bin_size]))
eps_mean_old = np.array(eps_mean_old)
eps_std_old = np.array(eps_std_old)/np.sqrt(bin_size)

eps_mean_new = []
eps_std_new = []

bins = len(eps_new_trap[0,:])//bin_size
for i in range(bins):
    eps_mean_new.append(np.mean(data3_filt_new.T[0,:][i*bin_size:(i+1)*bin_size]))
    eps_std_new.append(np.std(data3_filt_new.T[1,:][i*bin_size:(i+1)*bin_size]))
eps_mean_new = np.array(eps_mean_new)
eps_std_new = np.array(eps_std_new)/np.sqrt(bin_size)

eps_mean_new2 = []
eps_std_new2 = []

bins = len(eps_new_trap2[0,:])//bin_size
for i in range(bins):
    eps_mean_new2.append(np.mean(data3_filt_new2.T[0,:][i*bin_size:(i+1)*bin_size]))
    eps_std_new2.append(np.std(data3_filt_new2.T[1,:][i*bin_size:(i+1)*bin_size]))
eps_mean_new2 = np.array(eps_mean_new2)
eps_std_new2 = np.array(eps_std_new2)/np.sqrt(bin_size)

In [11]:
A_epsion = eps_mean_old
A_err_epsilon = eps_std_old
def calculate_pl(force_):
    chi2 = np.power((A_epsion-force_)/A_err_epsilon, 2)
    return np.sum(chi2)

A_epsion2 = eps_mean_new
A_err_epsilon2 = eps_std_new

A_epsion3 = eps_mean_new2
A_err_epsilon3 = eps_std_new2
def calculate_pl2(force_):
    chi2 = np.power((A_epsion2-force_)/A_err_epsilon2, 2)
    return np.sum(chi2)

In [12]:
np.mean(eps_std_new2), np.mean(eps_std_old)

(0.0034902815330926074, 0.00048732585222183822)

In [13]:
A_epsion = eps_mean_old
A_err_epsilon = eps_std_old

A_epsion2 = eps_mean_new
A_err_epsilon2 = eps_std_new

A_epsion3 = eps_mean_new2
A_err_epsilon3 = eps_std_new2

In [169]:
5e-4/(100*np.sqrt(10)*5)

3.1622776601683792e-07

In [17]:
def calculate_pl_combined_neutrality(force_):
    chi2 = np.sum(np.power((A_epsion-force_)/A_err_epsilon, 2))
    chi2 += np.sum(np.power((A_epsion2-force_)/A_err_epsilon2, 2))
    chi2 += np.sum(np.power((A_epsion3-force_)/A_err_epsilon3, 2))
    return np.sum(chi2)

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def calculate_pl_combined3(epsilon, density):
    
    electron = 1
    
    mu_N = density*6.23e23*420e-12

    if epsilon>=5e-4:
            res = 0
            for N1 in range(1):
                res_tmp = 0
                charge = epsilon*N1*electron
                res_tmp = np.power(mu_N,N1)*np.exp(-mu_N)/np.math.factorial(N1)
                res_tmp *= np.product(gaussian(A_epsion2, charge, A_err_epsilon2))
                for N2 in range(1):
                    charge2 = epsilon*N2
                    res_tmp2 = np.power(mu_N,N2)*np.exp(-mu_N)/np.math.factorial(N2)
                    res_tmp2 *= np.product(gaussian(A_epsion2, charge2, A_err_epsilon2))
                    for N3 in range(1):
                        charge3 = epsilon*N3
                        res_tmp3 = np.power(mu_N,N3)*np.exp(-mu_N)/np.math.factorial(N3)
                        res_tmp3 *= np.product(gaussian(A_epsion3, charge3, A_err_epsilon3))
                        res += res_tmp*res_tmp2*res_tmp3
#     elif (epsilon<5e-7)&(epsilon>=1e-7):
#             res = 0
#             for N1 in range(4):
#                 res_tmp = 0
#                 charge = epsilon*N1*electron
#                 res_tmp = np.power(mu_N,N1)*np.exp(-mu_N)/np.math.factorial(N1)
#                 res_tmp *= np.product(gaussian(A_epsion2, charge, A_err_epsilon2))
#                 for N2 in range(4):
#                     charge2 = epsilon*N2
#                     res_tmp2 = np.power(mu_N,N2)*np.exp(-mu_N)/np.math.factorial(N2)
#                     res_tmp2 *= np.product(gaussian(A_epsion2, charge2, A_err_epsilon2))
#                     res += res_tmp*res_tmp2
    elif (epsilon<5e-4)&(epsilon>=1e-5):
            res = 0
            for N1 in range(7):
                res_tmp = 0
                charge = epsilon*N1*electron
                res_tmp = np.power(mu_N,N1)*np.exp(-mu_N)/np.math.factorial(N1)
                res_tmp *= np.product(gaussian(A_epsion2, charge, A_err_epsilon2))
                for N2 in range(7):
                    charge2 = epsilon*N2
                    res_tmp2 = np.power(mu_N,N2)*np.exp(-mu_N)/np.math.factorial(N2)
                    res_tmp2 *= np.product(gaussian(A_epsion2, charge2, A_err_epsilon2))
#                     res += res_tmp*res_tmp2
                    for N3 in range(7):
                        charge3 = epsilon*N3
                        res_tmp3 = np.power(mu_N,N3)*np.exp(-mu_N)/np.math.factorial(N3)
                        res_tmp3 *= np.product(gaussian(A_epsion3, charge3, A_err_epsilon3))
                        res += res_tmp*res_tmp2*res_tmp3
                    
    elif (epsilon<1e-5)&(epsilon>=1e-6):
            res = 0
            for N1 in range(100):
                res_tmp = 0
                charge = epsilon*N1*electron
                res_tmp = np.power(mu_N,N1)*np.exp(-mu_N)/np.math.factorial(N1)
                res_tmp *= np.product(gaussian(A_epsion2, charge, A_err_epsilon2))
                for N2 in range(100):
                    charge2 = epsilon*N2
                    res_tmp2 = np.power(mu_N,N2)*np.exp(-mu_N)/np.math.factorial(N2)
                    res_tmp2 *= np.product(gaussian(A_epsion2, charge2, A_err_epsilon2))

#                     if res_tmp==0:
#                         print('res_tmp is zero at ', N1, charge, density)
#                         break
#                     if res_tmp2==0:
#                         print('res_tmp2 is zero at ', N1, charge2, density)
#                         break
                    res += res_tmp*res_tmp2
                    
    elif (epsilon<1e-6):
            res = 0
            for N1 in range(1):
                res_tmp = 0
                charge = epsilon*mu_N*electron
#                 res_tmp = np.power(mu_N,N1)*np.exp(-mu_N)/np.math.factorial(N1)
                res_tmp = np.product(gaussian(A_epsion2, charge, A_err_epsilon2))
                for N2 in range(1):
                    charge2 = epsilon*mu_N
#                     res_tmp2 = np.power(mu_N,N2)*np.exp(-mu_N)/np.math.factorial(N2)
                    res_tmp2 = np.product(gaussian(A_epsion2, charge2, A_err_epsilon2))
                    for N3 in range(1):
                        charge3 = epsilon*mu_N
                        res_tmp3 = np.product(gaussian(A_epsion3, charge3, A_err_epsilon3))
#                     if res_tmp==0:
#                         print('res_tmp is zero at ', N1, charge, density)
#                         break
#                     if res_tmp2==0:
#                         print('res_tmp2 is zero at ', N1, charge2, density)
#                         break
                    res += res_tmp*res_tmp2*res_tmp3
                    
    return -2*np.log(res)

In [15]:
density = 6e-17
mu_N = 6e23*75e-9*density
np.exp(-1*mu_N), mu_N, mu_N*1e-4

(0.067205512739749756, 2.7, 0.00027000000000000006)

In [199]:
2*8

16

In [181]:
## positive charge (opposing the electron charge)
## tuned for high mass

density_range = np.logspace(-18,-15,num=100)
density_range2 = np.logspace(-17,-14,num=100)
density_range3 = np.logspace(-16,-12,num=100)
density_range4 = np.logspace(-15,-11,num=100)
density_range5 = np.logspace(-17,-9,num=100)
# pl6 = np.array([calculate_pl_combined3(1e-5, pl_range_) for pl_range_ in density_range])
# pl7 = np.array([calculate_pl_combined3(1e-7, pl_range_) for pl_range_ in density_range])
# pl8 = np.array([calculate_pl_combined3(1e-8, pl_range_) for pl_range_ in density_range])
# pl9 = np.array([calculate_pl_combined3(1e-9, pl_range_) for pl_range_ in density_range2])
# pl10 = np.array([calculate_pl_combined3(1e-10, pl_range_) for pl_range_ in density_range3])
# pl11 = np.array([calculate_pl_combined3(1e-11, pl_range_) for pl_range_ in density_range4])

_,ax = plt.subplots()
ax.semilogx(density_range, pl6-np.min(pl6), 'o--', label=r'$10^{-5}$')
ax.semilogx(density_range, pl7-np.min(pl7), 'o--', label=r'$10^{-7}$')
ax.semilogx(density_range, pl8-np.min(pl8), 'o--', label=r'$10^{-8}$')
ax.semilogx(density_range2, pl9-np.min(pl9), 'o--', label=r'$10^{-9}$')
ax.semilogx(density_range3, pl10-np.min(pl10), 'o--', label=r'$10^{-10}$')
ax.semilogx(density_range4, pl11-np.min(pl11), 'o--', label=r'$10^{-11}$')
ax.plot(density_range5, density_range5*0+2.71, 'r--')
ax.set(ylim=(0,5), xlim=(1e-18,1e-10), xlabel='Abundance per nucleon, $n_{\chi}$', ylabel='PL')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [219]:
epsilon_limit = np.logspace(-24,0,num=100)
limit_projected = []
for eps_ in epsilon_limit:
    if eps_>1e-7:
        density_range = np.logspace(-18,-15,num=20)
    elif (eps_<1e-7)&(eps_>=1e-8):
        density_range = np.logspace(-18,-15,num=20)
    elif (eps_<1e-8)&(eps_>=1e-10):
        density_range = np.logspace(-16,-13,num=20)
    elif (eps_<1e-10)&(eps_>=1e-12):
        density_range = np.logspace(-15,-11,num=20)
    elif (eps_<1e-12)&(eps_>=1e-14):
        density_range = np.logspace(-13,-9,num=20)
    elif (eps_<1e-14)&(eps_>=1e-16):
        density_range = np.logspace(-11,-7,num=20)
    elif (eps_<1e-16)&(eps_>=1e-18):
        density_range = np.logspace(-9, -5,num=20)
    elif (eps_<1e-18)&(eps_>=1e-20):
        density_range = np.logspace(-7, -3,num=20)
    elif (eps_<1e-20)&(eps_>=1e-22):
        density_range = np.logspace(-5, -1,num=20)
    elif (eps_<1e-22)&(eps_>=1e-24):
        density_range = np.logspace(-3, 2,num=20)
        
    pl_tmp = np.array([calculate_pl_combined3(eps_, pl_range_) for pl_range_ in density_range])
    i = np.argmin(pl_tmp)
    limit_projected.append(np.interp(2.71, (pl_tmp-np.min(pl_tmp))[i:],density_range[i:]))
    print('limit at ', eps_, ' is ', np.interp(2.71, pl_tmp-np.min(pl_tmp),density_range))

limit at  1e-24  is  5.93093581244
limit at  1.74752840001e-24  is  3.38894006359
limit at  3.05385550883e-24  is  1.94085907322
limit at  5.33669923121e-24  is  1.11406880014
limit at  9.32603346883e-24  is  0.640932091803
limit at  1.62975083462e-23  is  0.369559313422
limit at  2.84803586844e-23  is  0.213557883683
limit at  4.97702356433e-23  is  0.12367831427
limit at  8.69749002618e-23  is  0.0707535709876
limit at  1.51991108295e-22  is  0.0401717675062
limit at  2.65608778295e-22  is  0.0233206073421
limit at  4.64158883361e-22  is  0.0131488657405
limit at  8.1113083079e-22  is  0.00745269257607
limit at  1.41747416293e-21  is  0.00424676996316
limit at  2.47707635599e-21  is  0.00243301821978
limit at  4.32876128108e-21  is  0.0014014257636
limit at  7.56463327555e-21  is  0.000811539291665
limit at  1.32194114847e-20  is  0.000455481667549
limit at  2.31012970008e-20  is  0.000261933751299
limit at  4.0370172586e-20  is  0.000151435489723
limit at  7.05480231072e-20  is  8.7

/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:104: RuntimeWarning: divide by zero encountered in log


 7.56463327555e-13  is  8.11539291665e-12
limit at  1.32194114847e-12  is  4.55481667549e-12
limit at  2.31012970008e-12  is  2.61933751299e-12
limit at  4.0370172586e-12  is  1.51435489723e-12
limit at  7.05480231072e-12  is  8.76504754621e-13
limit at  1.23284673944e-11  is  4.94456704881e-13
limit at  2.15443469003e-11  is  2.80404647344e-13
limit at  3.76493580679e-11  is  1.59870656032e-13
limit at  6.57933224658e-11  is  9.16436189335e-14
limit at  1.1497569954e-10  is  5.31524536098e-14
limit at  2.00923300257e-10  is  3.04753565903e-14
limit at  3.51119173422e-10  is  1.74435623104e-14
limit at  6.13590727341e-10  is  9.95850060703e-15
limit at  1.07226722201e-09  is  6.60689326167e-15
limit at  1.87381742286e-09  is  3.86518559407e-15
limit at  3.27454916288e-09  is  2.24429066487e-15
limit at  5.72236765935e-09  is  1.34969957886e-15
limit at  1e-08  is  6.14820746963e-16
limit at  1.74752840001e-08  is  3.58592945937e-16
limit at  3.05385550883e-08  is  2.38611129657e-16
lim

In [220]:
# ours = np.load('our_limit_extended.npy')
fig ,ax = plt.subplots()
ax.loglog(*ours, 'k-', label='limit on negatvie charge')
ax.loglog(epsilon_limit, limit_projected, 'k-', label='projection')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
## positive charge (opposing the electron charge)

density_range = np.logspace(-16,-13,num=100)
density_range2 = np.logspace(-15,-12,num=100)
density_range3 = np.logspace(-14,-11,num=100)
density_range4 = np.logspace(-12,-10,num=100)
density_range5 = np.logspace(-17,-9,num=100)
density_range6 = np.logspace(-2,1,num=100)
# pl6 = np.array([calculate_pl_combined3(1e-2, pl_range_) for pl_range_ in density_range])
# pl7 = np.array([calculate_pl_combined3(1e-4, pl_range_) for pl_range_ in density_range])
# pl8 = np.array([calculate_pl_combined3(1e-5, pl_range_) for pl_range_ in density_range])
# pl9 = np.array([calculate_pl_combined3(1e-6, pl_range_) for pl_range_ in density_range2])
# pl10 = np.array([calculate_pl_combined3(1e-7, pl_range_) for pl_range_ in density_range3])
# pl11 = np.array([calculate_pl_combined3(1e-8, pl_range_) for pl_range_ in density_range4])
pl12 = np.array([calculate_pl_combined3(2.6e-19, pl_range_) for pl_range_ in density_range6])

_,ax = plt.subplots()
# ax.semilogx(density_range, pl6-np.min(pl6), 'o--', label=r'$10^{-2}$')
# ax.semilogx(density_range, pl7-np.min(pl7), 'o--', label=r'$10^{-4}$')
# ax.semilogx(density_range, pl8-np.min(pl8), 'o--', label=r'$10^{-5}$')
# ax.semilogx(density_range2, pl9-np.min(pl9), 'o--', label=r'$10^{-6}$')
# ax.semilogx(density_range3, pl10-np.min(pl10), 'o--', label=r'$10^{-7}$')
# ax.semilogx(density_range4, pl11-np.min(pl11), 'o--', label=r'$10^{-8}$')
ax.semilogx(density_range6, pl12-np.min(pl12), 'o--', label=r'$10^{-8}$')
ax.plot(density_range6, density_range6*0+2.71, 'r--')
# ax.set(ylim=(0,5), xlim=(1e-16,1e-10), xlabel='Abundance per nucleon, $n_{\chi}$', ylabel='PL')
ax.set(ylim=(0,5), xlabel='Abundance per nucleon, $n_{\chi}$', ylabel='PL')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [336]:
i = np.argmin(pl10)
np.interp(2.71, (pl10-np.min(pl10))[i:],density_range[i:]), i

(2.7881807378506607e-14, 0)

In [209]:
## negative charge (like electron)

density_range = np.logspace(-16,-13,num=100)
density_range2 = np.logspace(-15,-12,num=100)
density_range3 = np.logspace(-16,-9,num=100)
density_range4 = np.logspace(-12,-10,num=100)
density_range5 = np.logspace(-17,-9,num=100)
pl6 = np.array([calculate_pl_combined3(1e-2, pl_range_) for pl_range_ in density_range])
pl7 = np.array([calculate_pl_combined3(1e-4, pl_range_) for pl_range_ in density_range])
pl8 = np.array([calculate_pl_combined3(1e-5, pl_range_) for pl_range_ in density_range])
pl9 = np.array([calculate_pl_combined3(1e-6, pl_range_) for pl_range_ in density_range2])
pl10 = np.array([calculate_pl_combined3(1e-7, pl_range_) for pl_range_ in density_range3])
pl11 = np.array([calculate_pl_combined3(1e-8, pl_range_) for pl_range_ in density_range4])

_,ax = plt.subplots()
ax.semilogx(density_range, pl6-np.min(pl6), 'o--', label=r'$10^{-2}$')
ax.semilogx(density_range, pl7-np.min(pl7), 'o--', label=r'$10^{-4}$')
ax.semilogx(density_range, pl8-np.min(pl8), 'o--', label=r'$10^{-5}$')
ax.semilogx(density_range2, pl9-np.min(pl9), 'o--', label=r'$10^{-6}$')
ax.semilogx(density_range3, pl10-np.min(pl10), 'o--', label=r'$10^{-7}$')
ax.semilogx(density_range4, pl11-np.min(pl11), 'o--', label=r'$10^{-8}$')
ax.plot(density_range5, density_range5*0+2.71, 'r--')
ax.set(ylim=(0,5), xlim=(1e-16,1e-10), xlabel='Abundance per nucleon, $n_{\chi}$', ylabel='PL')
ax.legend()

/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:261: RuntimeWarning: divide by zero encountered in log


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [156]:
epsilon_limit = np.logspace(-22,0,num=500)
limit = []
for eps_ in epsilon_limit:
    if eps_>1e-3:
        density_range = np.logspace(-15,-13,num=20)
    elif (eps_<1e-3)&(eps_>=1e-4):
        density_range = np.logspace(-15,-13,num=20)
    elif (eps_<1e-4)&(eps_>=1e-5):
        density_range = np.logspace(-15,-13,num=20)
    elif (eps_<1e-5)&(eps_>=1e-6):
        density_range = np.logspace(-15,-13,num=10)
    elif (eps_<1e-6)&(eps_>=1e-8):
        density_range = np.logspace(-16,-12,num=20)
    elif (eps_<1e-8)&(eps_>=1e-10):
        density_range = np.logspace(-15,-9,num=20)
    elif (eps_<1e-10)&(eps_>=1e-12):
        density_range = np.logspace(-12,-8,num=20)
    elif (eps_<1e-12)&(eps_>=1e-14):
        density_range = np.logspace(-12,-6,num=20)
    elif (eps_<1e-14)&(eps_>=1e-16):
        density_range = np.logspace(-10,-4,num=20)
    elif (eps_<1e-16)&(eps_>=1e-18):
        density_range = np.logspace(-8, -2,num=20)
    elif (eps_<1e-18)&(eps_>=1e-20):
        density_range = np.logspace(-6, 0,num=20)
    elif (eps_<1e-20)&(eps_>=1e-22):
        density_range = np.logspace(-6, 2,num=20)
    elif (eps_<1e-22)&(eps_>=1e-24):
        density_range = np.logspace(-6, 2,num=20)
        
    pl_tmp = np.array([calculate_pl_combined3(eps_, pl_range_) for pl_range_ in density_range])
    i = np.argmin(pl_tmp)
    limit.append(np.interp(2.71, (pl_tmp-np.min(pl_tmp))[i:],density_range[i:]))
    print('limit at ', eps_, ' is ', np.interp(2.71, pl_tmp-np.min(pl_tmp),density_range))

limit at  1e-22  is  100.0
limit at  1.10684848855e-22  is  100.0
limit at  1.2251135766e-22  is  100.0
limit at  1.35601511057e-22  is  100.0
limit at  1.50090327558e-22  is  100.0
limit at  1.66127252203e-22  is  100.0
limit at  1.83877698008e-22  is  100.0
limit at  2.03524752118e-22  is  100.0
limit at  2.25271064265e-22  is  100.0
limit at  2.49340936995e-22  is  100.0
limit at  2.75982639247e-22  is  100.0
limit at  3.05470967116e-22  is  100.0
limit at  3.38110078248e-22  is  100.0
limit at  3.74236629072e-22  is  90.9705524593
limit at  4.14223247248e-22  is  79.9051340199
limit at  4.58482375139e-22  is  70.6417698835
limit at  5.07470523949e-22  is  62.9091903284
limit at  5.61692982416e-22  is  56.4725810437
limit at  6.21709028616e-22  is  51.1295737089
limit at  6.88137698642e-22  is  46.706499563
limit at  7.61664171655e-22  is  43.0549240259
limit at  8.43046837179e-22  is  40.0484725132
limit at  9.33125117508e-22  is  37.1452589003
limit at  1.03282812594e-21  is  32.5

/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:98: RuntimeWarning: divide by zero encountered in log


 1.43984795288e-17  is  0.00239523625585
limit at  1.59369353038e-17  is  0.00215053992177
limit at  1.76397727531e-17  is  0.00190459977474
limit at  1.95245558102e-17  is  0.00169922644447
limit at  2.16107250881e-17  is  0.00152822610093
limit at  2.39197984002e-17  is  0.00138625363787
limit at  2.64755927057e-17  is  0.0012687161942
limit at  2.93044697697e-17  is  0.00117168360728
limit at  3.24356080724e-17  is  0.00106076087451
limit at  3.59013037701e-17  is  0.000938396479162
limit at  3.97373038149e-17  is  0.000836166859203
limit at  4.39831746665e-17  is  0.00075100722624
limit at  4.86827104012e-17  is  0.000680270860384
limit at  5.38843844261e-17  is  0.000621681959227
limit at  5.96418494584e-17  is  0.000573291802318
limit at  6.60144909273e-17  is  0.000523313207338
limit at  7.30680395053e-17  is  0.000462437079036
limit at  8.08752490877e-17  is  0.000411553654267
limit at  8.95166472138e-17  is  0.000369146615071
limit at  9.90813656686e-17  is  0.000333905543481


limit at  4.80134292365e-08  is  7.1010501237e-13
limit at  5.31435915805e-08  is  6.48456314476e-13
limit at  5.8821904017e-08  is  5.88390599509e-13
limit at  6.51069355548e-08  is  5.24940565557e-13
limit at  7.20635132129e-08  is  4.72071923459e-13
limit at  7.97633906793e-08  is  4.28149637813e-13
limit at  8.8285988415e-08  is  3.91766387745e-13
limit at  9.77192128372e-08  is  3.52954288332e-13
limit at  1.08160363031e-07  is  3.15439143886e-13
limit at  1.19717134342e-07  is  2.84201017963e-13
limit at  1.325087292e-07  is  2.58266003556e-13
limit at  1.46667086634e-07  is  2.3679663997e-13
limit at  1.62338243161e-07  is  2.1180712785e-13
limit at  1.79683839077e-07  is  1.89629197643e-13
limit at  1.98882785699e-07  is  1.71174254758e-13
limit at  2.20133110749e-07  is  1.55862319837e-13
limit at  2.43654000913e-07  is  1.4287445592e-13
limit at  2.69688062639e-07  is  1.2715677023e-13
limit at  2.98503824512e-07  is  1.14047502092e-13
limit at  3.30398506987e-07  is  1.03146

In [223]:
from scipy.signal import savgol_filter
yhat2 = savgol_filter(limit_projected, 5, 3) # window size 51, polynomial order 3

In [225]:
# ours = np.load('our_limit_extended.npy')

fig ,ax = plt.subplots()
ax.loglog(*ours, 'k-', label='limit on negatvie charge')
ax.loglog(epsilon_limit, limit_projected, 'k-', label='projection')
# ax.loglog(epsilon_sensitivity, smoothed_sensitivity, 'k-', label='limit on positive charge')
# ax.loglog(*our_data, 'k-', label='limit on negatvie charge')
# ax.loglog(gadi_data[::2], gadi_data[1::2], '--', label='Yale')
# ax.loglog(dave_data[::2], dave_data[1::2], '--', label='Stanford 2014')
# ax.set(xlabel='Fractional charge, $\epsilon$', ylabel='Abundance per nucleon, $n_{\chi}$', ylim=(1e-20,1e-6),xlim=(1e-12,1.01))
ax.legend()
# fig.savefig('fig4_limit_millicharge.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [226]:
our_data = np.array([epsilon_limit, limit_projected])
np.save('our_projection_extended_high_mass', our_data)

In [79]:
gadi_data = np.array([0.9896468028945953, 2.2238599725966594e-17,
0.19516209893399206, 2.3997460707811468e-17,
0.0093459072975982, 2.3545115567287907e-17,
0.002625481966069156, 2.5407309482131215e-17,
0.0011659144011798323, 4.0885587995589997e-17,
0.0006052306368592101, 7.661212713002802e-17,
0.000280193223174577, 1.7037464293995588e-16,
0.0001133019248813323, 4.496685997870468e-16,
0.00004629528085832118, 1.1868071806067058e-15,
0.00004629528085832118, 1.0192118462630151e-8])

In [90]:
dave_data = np.array([0.9849491981099865, 3.580818436386276e-15,
0.12522609583772226, 3.916700414732544e-15,
0.013474975716778723, 3.916700414732544e-15,
0.0014499770942557823, 3.916700414732544e-15,
0.00032803227444906034, 4.685937159057862e-15,
0.00017884212165757526, 6.132119639330011e-15,
0.00009459119530973852, 1.2563627290624282e-14,
0.00004853547402209772, 2.8155129039539425e-14,
0.00004299024529768444, 4.8215407398725386e-14,
0.00004299024529768444, 8.495785518249188e-13,
0.00004299024529768444, 0.0000018884530615508627,
0.00004299024529768444, 0.0009179771233537134,
0.00004299024529768444, 0.6387188517346809])

In [55]:
high_charge_limit = 2.726e-15
q_corner_one_sphere = 9.75e-5

integration_time_hours = 90
number_of_spheres = 1
noise_floor_factor = 10
electric_field_factor = 2

high_limit = high_charge_limit/number_of_spheres/8
q_corner = 9.75e-5/np.sqrt(90/integration_time_hours*number_of_spheres)/noise_floor_factor/electric_field_factor

sensitivity = []
epsilon_sensitivity = np.logspace(-18.7,0,num=200)
for eps_ in epsilon_sensitivity:
    if eps_>q_corner:
        sensitivity.append(high_limit)
    else:
        sensitivity.append(q_corner/eps_*high_limit)
        
from scipy.signal import savgol_filter
smoothed_sensitivity = savgol_filter(sensitivity, 21, 3) # window size 51, polynomial order 3

In [272]:
_,ax = plt.subplots()
pl_range = np.arange(-2e-4,1e-4,1e-5)
pl = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range])
ax.plot(pl_range*100000, pl-np.min(pl), 'o--', label='-2log(likelihood)')
ax.plot(pl_range*100000, pl_range*0+2.7, 'r--')
ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL',)

pl_range_plus = np.arange(0,1e-4,1e-5)
pl_plus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_plus])
ax.plot(pl_range_plus*100000, pl_plus-np.min(pl_plus), 'o--', label='profile likelihood')

pl_range_minus = np.arange(-2e-4,pl_range[np.argmin(pl)],1e-5)
pl_minus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_minus])
ax.plot(pl_range_minus*100000, pl_minus-np.min(pl_minus), 'o--', label='profile likelihood')
ax.set(xlim=(-15,8), ylim=(-0.1,6))
ax.legend()
print('mean ', pl_range[np.argmin(pl)])
print('limit ', np.interp(2.71, (pl_minus-np.min(pl_minus))[::-1], pl_range_minus[::-1]), np.interp(2.71, pl_plus-np.min(pl_plus), pl_range_plus))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean  2e-05
limit  -4.41486558343e-05 8.44962290276e-05


In [ ]:
_,ax = plt.subplots()
pl_range_plus = np.arange(0,1e-4,1e-5)
pl_plus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_plus])
ax.plot(pl_range_plus*100000, pl_plus-np.min(pl_plus), 'o--')
ax.plot(pl_range_plus*100000, pl_range_plus*0+2.7, 'r--')

ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')
pl_range[np.argmin(pl)], np.interp(2.71, pl_plus-np.min(pl_plus), pl_range_plus)

In [ ]:
_,ax = plt.subplots()
pl_range_minus = np.arange(-2e-4,pl_range[np.argmin(pl)],1e-5)
pl_minus = np.array([calculate_pl(pl_range_) for pl_range_ in pl_range_minus])
ax.plot(pl_range_minus*100000, pl_minus-np.min(pl_minus), 'o--')
ax.plot(pl_range_minus*100000, pl_range_minus*0+2.7, 'r--')

ax.set(xlabel=r'$\epsilon [10^5]$', ylabel='PL')
pl_range[np.argmin(pl)], np.interp(2.71, (pl_minus-np.min(pl_minus))[::-1], pl_range_minus[::-1])